In [ ]:
%pip install ultralytics



In [3]:
import os
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from ultralytics import YOLO
import glob

In [4]:
INPUT_DIR = r"C:/Users/user/Documents/AI_project_clone/data_set_helmet/css-data"
WORK_DIR = r"C:/Users/user/Documents/AI_project_clone//working"

In [4]:
import os

# 데이터 경로 설정
LABELS_DIR = os.path.join(INPUT_DIR, 'train/labels')  # train 라벨 디렉토리
ALLOWED_CLASSES = {'0': 'helmet', '2': 'No-helmet', '5': 'person'}  # 유지할 클래스 ID
CLASS_MAPPING = {'0': '0', '2': '1', '5': '2'}  # 새로운 클래스 ID 매핑

def update_labels(labels_dir):
    for file in os.listdir(labels_dir):
        file_path = os.path.join(labels_dir, file)
        
        updated_lines = []
        with open(file_path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                cls_id, *coords = line.split()
                if cls_id in ALLOWED_CLASSES:  # 허용된 클래스만 유지
                    new_cls_id = CLASS_MAPPING[cls_id]  # 클래스 ID 재매핑
                    updated_lines.append(f"{new_cls_id} " + " ".join(coords) + "\n")
        
        # 라벨 파일 업데이트
        with open(file_path, 'w') as f:
            f.writelines(updated_lines)

# train, valid, test 디렉토리의 라벨 파일 수정
for mode in ['train', 'valid', 'test']:
    labels_dir = os.path.join(INPUT_DIR, f"{mode}/labels")
    update_labels(labels_dir)

print("라벨 파일 수정이 완료되었습니다.")


라벨 파일 수정이 완료되었습니다.


In [ ]:


# WORK_DIR 디렉토리 생성
os.makedirs(WORK_DIR, exist_ok=True)

# 수정된 클래스 설정
num_classes = 3
classes = ['helmet', 'No-helmet', 'Person']  # Hardhat -> helmet, NO-Hardhat -> No-helmet

# data.yaml 파일 생성
dict_file = {
    'train': os.path.join(INPUT_DIR, 'train/images'),
    'val': os.path.join(INPUT_DIR, 'valid/images'),
    'test': os.path.join(INPUT_DIR, 'test/images'),
    'nc': num_classes,
    'names': classes
}

# data.yaml 저장
data_yaml_path = os.path.join(WORK_DIR, 'data.yaml')
with open(data_yaml_path, 'w+') as file:
    yaml.dump(dict_file, file)

print(f"'data.yaml' 파일이 '{data_yaml_path}'에 저장되었습니다.")

# 라벨 데이터 수정 (Hardhat -> helmet, NO-Hardhat -> No-helmet, Person 그대로, 나머지 제거)
ALLOWED_CLASSES = {'0': 'helmet', '2': 'No-helmet', '5': 'Person'}  # 기존 클래스 ID 매핑
CLASS_MAPPING = {'0': '0', '2': '1', '5': '2'}  # 새로운 클래스 ID 매핑


def update_labels(labels_dir):
    for file in os.listdir(labels_dir):
        file_path = os.path.join(labels_dir, file)
        
        updated_lines = []
        with open(file_path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                cls_id, *coords = line.split()
                if cls_id in ALLOWED_CLASSES:  # 허용된 클래스만 유지
                    new_cls_id = CLASS_MAPPING[cls_id]  # 클래스 ID 재매핑
                    updated_lines.append(f"{new_cls_id} " + " ".join(coords) + "\n")
        
        # 라벨 파일 업데이트
        with open(file_path, 'w') as f:
            f.writelines(updated_lines)


# train, valid, test 디렉토리의 라벨 파일 수정
for mode in ['train', 'valid', 'test']:
    labels_dir = os.path.join(INPUT_DIR, f"{mode}/labels")
    update_labels(labels_dir)

print("라벨 파일 수정이 완료되었습니다.")

# 데이터셋 크기 출력
for mode in ['train', 'valid', 'test']:
    files = glob.glob(os.path.join(INPUT_DIR, mode, 'images', '*'))
    print(f'{mode} set size: {len(files)}\n')

# YOLOv8 모델 학습
model = YOLO('yolov8n.pt')

model.train(
    data=data_yaml_path,
    task='detect',
    imgsz=640,
    epochs=30,
    batch=16,
    mode='train',
    name='yolov8n_v1_train'
)

print("Training completed!")


In [ ]:
from ultralytics import YOLO
import cv2

# 학습된 모델 로드
model = YOLO('runs/detect/yolov8n_v1_train/weights/best.pt')  # 학습 완료된 모델 경로

# OpenCV를 사용하여 웹캠 열기
cap = cv2.VideoCapture(0)  # 0은 기본 카메라. 다른 카메라를 사용하려면 1 또는 다른 번호로 변경

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# 카메라에서 실시간 객체 탐지
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    # YOLO 모델로 프레임 추론
    results = model.predict(source=frame, save=False, show=False)

    # 탐지 결과를 그리기
    annotated_frame = results[0].plot()  # 결과를 그린 프레임

    # 화면에 표시
    cv2.imshow('YOLOv8 Real-time Detection', annotated_frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()


In [ ]:
%pip install ultralytics

## 데이터 학습코드

In [ ]:
from ultralytics import YOLO

# 데이터 경로 및 설정
data_yaml_path = r"C:\Users\user\Documents\AI_project_clone\dataset\data.yaml"  # YAML 파일 경로 (수정 필요)
model_checkpoint = 'yolov8n.pt'  # YOLOv8 Nano 모델 가중치 (기본값)

# YOLO 모델 초기화
model = YOLO(model_checkpoint)


# 모델 학습
model.train(
    data=data_yaml_path,
    epochs=50,
    imgsz=640,
    batch=16,
    lr0=0.005,  # 초기 학습률 설정
    name='helmet_detection_model',
    project='helmet_detection'
)


print("학습완료")


## 로그인창//눈 깜빡임 2번 인식(추가로 해야함)
## +추가로 선입력으로 눈 두번 깜빡이면 객체 인식시작

In [ ]:
%pip install mediapipe

In [ ]:
import cv2
import time
import mediapipe as mp
from ultralytics import YOLO

# YOLO 모델 로드
model = YOLO(r"C:\Users\user\Documents\AI_project_clone\helmet_detection\helmet_detection_model\weights\best.pt")

# Mediapipe Face Mesh 초기화
face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)

# 웹캠 열기
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# 눈 깜빡임 감지 상태 변수 초기화
blink_count = 0  # 깜빡임 횟수
last_blink_time = None  # 마지막 깜빡임 시간
BLINK_DELAY = 2  # 눈 깜빡임 간격 제한 (초)

# 헬멧 인식 활성화 플래그
helmet_detection_mode = False

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    frame = cv2.flip(frame, 1)  # 좌우 반전
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Mediapipe로 얼굴 인식 및 눈 깜빡임 감지
    output = face_mesh.process(rgb_frame)
    landmark_points = output.multi_face_landmarks
    frame_h, frame_w, _ = frame.shape

    eye_closed = False

    if landmark_points:
        landmarks = landmark_points[0].landmark
        left_eye = [landmarks[145], landmarks[159]]
        for landmark in left_eye:
            x = int(landmark.x * frame_w)
            y = int(landmark.y * frame_h)
            cv2.circle(frame, (x, y), 3, (0, 255, 255))  # 왼쪽 눈 표시

        # 왼쪽 눈 깜빡임 감지
        if abs(left_eye[0].y - left_eye[1].y) < 0.005:
            eye_closed = True

        # 눈 깜빡임 로직
        if eye_closed and (last_blink_time is None or time.time() - last_blink_time > BLINK_DELAY):
            blink_count += 1
            last_blink_time = time.time()  # 마지막 깜빡임 시간 기록

    # 두 번 연속 깜빡임 감지 시 헬멧 인식 활성화
    if blink_count >= 2:
        helmet_detection_mode = True
        blink_count = 0  # 깜빡임 카운트 초기화

    if helmet_detection_mode:
        # YOLO로 헬멧 탐지
        results = model.predict(source=frame, save=False, show=False)

        for box in results[0].boxes:
            cls = int(box.cls)
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])

            # 바운딩 박스 그리기
            if cls == 0:  # No-helmet
                color = (0, 0, 255)  # 빨간색
                label = "No helmet"
            elif cls == 1:  # With-helmet
                color = (0, 255, 0)  # 초록색
                label = "With helmet"
            

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(
                frame,
                f"{label} ({conf:.2f})",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                color,
                2,
            )

        # 화면 상단 메시지
        cv2.putText(frame, "Helmet detection mode", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)

    else:
        # 일반 모드에서 메시지 표시
        cv2.putText(frame, "Blink twice to start helmet detection", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 2)

    # 결과 화면 표시
    cv2.imshow("Helmet and Eye Detection", frame)

    # 'q' 키로 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


## 눈깜빡임이랑 헬멧을 같이 써야인식

In [ ]:
import cv2
import time
import mediapipe as mp
from ultralytics import YOLO

# YOLO 모델 로드
model = YOLO(r"C:\Users\user\Documents\AI_project_clone\helmet_detection\helmet_detection_model\weights\best.pt")

# Mediapipe Face Mesh 초기화
face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)

# 웹캠 열기
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()


# 헬멧 감지 및 로그인 상태 변수 초기화
helmet_detected_time = None
eye_blink_detected = False
login_shown = False

# 로그인 창 표시 함수
def show_login_window():
    global login_shown
    if not login_shown:
        from tkinter import Tk, simpledialog
        root = Tk()
        root.withdraw()
        simpledialog.askstring("Login", "Please log in to continue:")
        login_shown = True
        root.destroy()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    frame = cv2.flip(frame, 1)  # 좌우 반전
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Mediapipe로 얼굴 인식 및 눈 깜빡임 감지
    output = face_mesh.process(rgb_frame)
    landmark_points = output.multi_face_landmarks
    frame_h, frame_w, _ = frame.shape

    if landmark_points:
        landmarks = landmark_points[0].landmark
        left_eye = [landmarks[145], landmarks[159]]
        for landmark in left_eye:
            x = int(landmark.x * frame_w)
            y = int(landmark.y * frame_h)
            cv2.circle(frame, (x, y), 3, (0, 255, 255))  # 왼쪽 눈 표시

        # 왼쪽 눈 깜빡임 감지
        if abs(left_eye[0].y - left_eye[1].y) < 0.005:
            eye_blink_detected = True  # 눈 깜빡임 감지
        else:
            eye_blink_detected = False

    # YOLO로 헬멧 탐지
    results = model.predict(source=frame, save=False, show=False)

    helmet_detected = False
    for box in results[0].boxes:
        cls = int(box.cls)
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf[0])

        # 바운딩 박스 그리기
        if cls == 0:  # No-helmet
            color = (0, 0, 255)  # 빨간색
            label = "No helmet"
            helmet_detected_time = None
        elif cls == 1:  # With-helmet
            color = (0, 255, 0)  # 초록색
            label = "With helmet"
            helmet_detected = True  # 헬멧 감지
            if helmet_detected_time is None:
                helmet_detected_time = time.time()

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(
            frame,
            f"{label} ({conf:.2f})",
            (x1, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            color,
            2,
        )

    # 눈 깜빡임과 헬멧 착용 감지가 모두 충족되었는지 확인
    if eye_blink_detected and helmet_detected and helmet_detected_time:
        if time.time() - helmet_detected_time >= 3:  # 3초 이상 헬멧 감지
            show_login_window()  # 로그인 창 표시

    # 화면 상단 메시지 표시
    if not helmet_detected:
        cv2.putText(frame, "Please wear a helmet & wink ", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)
    elif helmet_detected:
        cv2.putText(frame, "Helmet detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)

    # 결과 화면 표시
    cv2.imshow("Helmet and Eye Detection", frame)

    # 'q' 키로 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


## 모자데이터 포함한 모델: 모자와 헬멧 미착용시 로그인창 비활성화

In [ ]:
import cv2
import time
import mediapipe as mp
from ultralytics import YOLO

# YOLO 모델 로드
model = YOLO(r"C:\Users\user\Documents\AI_project_clone\helmet_detection\helmet_detection_model\weights\best.pt")

# Mediapipe Face Mesh 초기화
face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)

# 웹캠 열기
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# 눈 깜빡임 감지 상태 변수 초기화
blink_count = 0  # 깜빡임 횟수
last_blink_time = None  # 마지막 깜빡임 시간
BLINK_DELAY = 2  # 눈 깜빡임 간격 제한 (초)

# 헬멧 감지 상태 변수 초기화
helmet_detection_mode = False
helmet_detected_time = None
login_shown = False

# 로그인 창 표시 함수
def show_login_window():
    global login_shown
    if not login_shown:
        from tkinter import Tk, simpledialog
        root = Tk()
        root.withdraw()
        simpledialog.askstring("Login", "Please log in to continue:")
        login_shown = True
        root.destroy()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    frame = cv2.flip(frame, 1)  # 좌우 반전
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Mediapipe로 얼굴 인식 및 눈 깜빡임 감지
    output = face_mesh.process(rgb_frame)
    landmark_points = output.multi_face_landmarks
    frame_h, frame_w, _ = frame.shape

    eye_closed = False

    if landmark_points:
        landmarks = landmark_points[0].landmark
        left_eye = [landmarks[145], landmarks[159]]
        for landmark in left_eye:
            x = int(landmark.x * frame_w)
            y = int(landmark.y * frame_h)
            cv2.circle(frame, (x, y), 3, (0, 255, 255))  # 왼쪽 눈 표시

        # 왼쪽 눈 깜빡임 감지
        if abs(left_eye[0].y - left_eye[1].y) < 0.005:
            eye_closed = True

        # 눈 깜빡임 로직
        if eye_closed and (last_blink_time is None or time.time() - last_blink_time > BLINK_DELAY):
            blink_count += 1
            last_blink_time = time.time()  # 마지막 깜빡임 시간 기록

    # 두 번 연속 깜빡임 감지 시 헬멧 인식 활성화
    if blink_count >= 2:
        helmet_detection_mode = True
        blink_count = 0  # 깜빡임 카운트 초기화

    if helmet_detection_mode:
        # YOLO로 헬멧 탐지
        results = model.predict(source=frame, save=False, show=False)

        for box in results[0].boxes:
            cls = int(box.cls)
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])

            # 바운딩 박스 그리기
            if cls == 0:  # No-helmet
                color = (0, 0, 255)  # 빨간색
                label = "No helmet"
                helmet_detected_time = None  # 타이머 초기화
                login_shown = False  # 로그인 비활성화

            elif cls == 1:  # With-helmet
                color = (0, 255, 0)  # 초록색
                label = "With helmet"
                if helmet_detected_time is None:
                    helmet_detected_time = time.time()  # 헬멧 감지 시작 시간 기록
                elif time.time() - helmet_detected_time >= 3 and not login_shown:
                    show_login_window()  # 헬멧 감지 3초 후 로그인 창 표시

            elif cls == 2:  # With cap
                color = (0, 255, 255)  # 노란색
                label = "With cap"
                helmet_detected_time = None  # 타이머 초기화
                login_shown = False  # 로그인 비활성화

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(
                frame,
                f"{label} ({conf:.2f})",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                color,
                2,
            )

    # 메시지 표시
    if helmet_detection_mode:
        cv2.putText(frame, "Helmet detection mode", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)
    else:
        cv2.putText(frame, "Blink twice to start helmet detection", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 2)

    # 결과 화면 표시
    cv2.imshow("Helmet and Eye Detection", frame)

    # 'q' 키로 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


## 착용시간이 3초 지속되면 qr 코드가 켜지도록

In [ ]:
import cv2
import time
from ultralytics import YOLO

# QR 코드 이미지 경로
qr_image_path = r"C:\Users\user\Documents\AI_project_clone\QR.png" # 생성된 QR 코드 이미지 파일 경로

# 학습된 모델 로드
model = YOLO(r"C:\Users\user\Documents\AI_project_clone\helmet_detection\helmet_detection_model\weights\best.pt")

# OpenCV를 사용하여 웹캠 열기
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# QR 코드 이미지 로드
qr_image = cv2.imread(qr_image_path)
if qr_image is None:
    print(f"Error: Could not load QR code image from {qr_image_path}")
    exit()

# QR 코드 표시 여부 플래그와 헬멧 상태 타이머 초기화
show_qr_code = False
helmet_detected_time = None  # 헬멧이 처음 감지된 시간을 저장

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    # YOLO 모델로 프레임 추론
    results = model.predict(source=frame, save=False, show=False)

    # 헬멧 착용 여부 확인 변수
    helmet_detected = False
    no_helmet_detected = False

    # 탐지 결과 처리
    for box in results[0].boxes:
        cls = int(box.cls)  # 클래스 ID
        conf = float(box.conf[0])  # 신뢰도
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # 바운딩 박스 좌표

        if cls == 0:  # No-helmet
            label = "No helmet"
            no_helmet_detected = True  # 헬멧 미착용 상태 감지
            color = (0, 0, 255)  # 빨간색
            helmet_detected_time = None  # 헬멧 감지 타이머 초기화
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

        elif cls == 1:  # With-helmet
            label = "With helmet"
            helmet_detected = True  # 헬멧 착용 상태 감지
            color = (0, 255, 0)  # 초록색
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

        # 라벨 텍스트 표시
        cv2.putText(
            frame,
            f"{label} ({conf:.2f})",
            (x1, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            color,
            2,
        )

    # 헬멧 감지 타이머 로직
    if helmet_detected:
        if helmet_detected_time is None:
            helmet_detected_time = time.time()  # 처음 헬멧이 감지된 시간 기록
        elif time.time() - helmet_detected_time >= 3:  # 3초 연속 감지 확인
            # QR 코드 표시
            if not show_qr_code:
                print("Displaying QR Code. Scan to visit the website.")
                cv2.imshow("QR Code", qr_image)
                show_qr_code = True

    else:
        helmet_detected_time = None  # 헬멧 감지 리셋
        if show_qr_code:
            cv2.destroyWindow("QR Code")
            show_qr_code = False

    # 화면 상단에 메시지 표시
    if no_helmet_detected:
        cv2.putText(frame, "Please wear a helmet", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)

    elif helmet_detected:
        cv2.putText(frame, "Helmet detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)

    # 결과 화면 표시
    cv2.imshow("Helmet Detection", frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


## 헬멧 착용 인식후에 qr 창 출력(10초후 qr창 제거)

In [ ]:
import cv2
import time
import mediapipe as mp
from ultralytics import YOLO


# YOLO 모델 로드
model = YOLO(r"C:\Users\user\Documents\AI_project_clone\helmet_detection\helmet_detection_model\weights\best.pt")

# Mediapipe Face Mesh 초기화
face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)

# QR 코드 이미지 경로
qr_image_path = r"C:\Users\user\Documents\AI_project_clone\QR.png"
qr_image = cv2.imread(qr_image_path)  # QR 코드 이미지 로드
qr_image = cv2.resize(qr_image,(300,300))
# 웹캠 열기
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# 눈 깜빡임 감지 상태 변수 초기화
blink_count = 0  # 깜빡임 횟수
last_blink_time = None  # 마지막 깜빡임 시간
BLINK_DELAY = 2  # 눈 깜빡임 간격 제한 (초)

# 헬멧 감지 상태 변수 초기화
helmet_detection_mode = False
helmet_detected_time = None
qr_displayed_time = None

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    frame = cv2.flip(frame, 1)  # 좌우 반전
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Mediapipe로 얼굴 인식 및 눈 깜빡임 감지
    output = face_mesh.process(rgb_frame)
    landmark_points = output.multi_face_landmarks
    frame_h, frame_w, _ = frame.shape

    eye_closed = False

    if landmark_points:
        landmarks = landmark_points[0].landmark
        left_eye = [landmarks[145], landmarks[159]]
        for landmark in left_eye:
            x = int(landmark.x * frame_w)
            y = int(landmark.y * frame_h)
            cv2.circle(frame, (x, y), 3, (0, 255, 255))  # 왼쪽 눈 표시

        # 왼쪽 눈 깜빡임 감지
        if abs(left_eye[0].y - left_eye[1].y) < 0.005:
            eye_closed = True

        # 눈 깜빡임 로직
        if eye_closed and (last_blink_time is None or time.time() - last_blink_time > BLINK_DELAY):
            blink_count += 1
            last_blink_time = time.time()  # 마지막 깜빡임 시간 기록

    # 두 번 연속 깜빡임 감지 시 헬멧 인식 활성화
    if blink_count >= 2:
        helmet_detection_mode = True
        blink_count = 0  # 깜빡임 카운트 초기화

    if helmet_detection_mode:
        # YOLO로 헬멧 탐지
        results = model.predict(source=frame, save=False, show=False)

        for box in results[0].boxes:
            cls = int(box.cls)
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])

            # 바운딩 박스 그리기
            if cls == 0:  # No-helmet
                color = (0, 0, 255)  # 빨간색
                label = "No helmet"
                helmet_detected_time = None  # 타이머 초기화

            elif cls == 1:  # With-helmet
                color = (0, 255, 0)  # 초록색
                label = "With helmet"
                if helmet_detected_time is None:
                    helmet_detected_time = time.time()  # 헬멧 감지 시작 시간 기록
                elif time.time() - helmet_detected_time >= 3 and qr_displayed_time is None:
                    qr_displayed_time = time.time()  # QR 코드 표시 시작 시간
                    cv2.imshow("Scan QR Code", qr_image)  # QR 코드 표시

            elif cls == 2 and conf >= 0.8:  # With cap
                
                color = (0, 255, 255)  # 노란색
                label = "With cap"
                helmet_detected_time = None  # 타이머 초기화
                # "Please wear a helmet" 메시지 표시
                cv2.putText(frame, "Please wear a helmet", (frame_w // 4, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(
                frame,
                f"{label} ({conf:.2f})",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                color,
                2,
            )

        # QR 코드 표시 시간 확인
        if qr_displayed_time and time.time() - qr_displayed_time > 10:  # 10초 이후 QR 코드 닫기
            qr_displayed_time = None
            cv2.destroyWindow("Scan QR Code")

    # 메시지 표시
    if helmet_detection_mode:
        cv2.putText(frame, "Helmet detection mode", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)
    else:
        cv2.putText(frame, "Blink twice to start helmet detection", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 2)

    # 결과 화면 표시
    cv2.imshow("Helmet and Eye Detection", frame)

    # 'q' 키로 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


## 최종

In [ ]:
import cv2
import time
import mediapipe as mp
from ultralytics import YOLO
import os

# YOLO 모델 로드
model = YOLO(r"C:\Users\user\Documents\AI_project_clone\helmet_detection\helmet_detection_model\weights\best.pt")

# Mediapipe Face Mesh 초기화
face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)

# QR 코드 이미지 경로
qr_image_path = r"C:\Users\user\Documents\AI_project_clone\사용자인증.jpeg"
qr_image = cv2.imread(qr_image_path)  # QR 코드 이미지 로드
qr_image = cv2.resize(qr_image, (300, 300))

# 다운로드된 Streamlit 파일 경로
streamlit_file_path = r"C:\Users\user\Downloads\streamlit_download.txt"

# 웹캠 열기
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# 눈 깜빡임 감지 상태 변수 초기화
blink_count = 0  # 깜빡임 횟수
last_blink_time = None  # 마지막 깜빡임 시간
BLINK_DELAY = 2  # 눈 깜빡임 간격 제한 (초)

# 헬멧 감지 상태 변수 초기화
helmet_detection_mode = False
helmet_detected_time = None
qr_displayed_time = None
authentication_complete_time = None
authentication_completed = False  # 인증 완료 상태 플래그

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    frame = cv2.flip(frame, 1)  # 좌우 반전
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Mediapipe로 얼굴 인식 및 눈 깜빡임 감지
    output = face_mesh.process(rgb_frame)
    landmark_points = output.multi_face_landmarks
    frame_h, frame_w, _ = frame.shape

    eye_closed = False

    if landmark_points:
        landmarks = landmark_points[0].landmark
        left_eye = [landmarks[145], landmarks[159]]
        for landmark in left_eye:
            x = int(landmark.x * frame_w)
            y = int(landmark.y * frame_h)
            cv2.circle(frame, (x, y), 3, (0, 255, 255))  # 왼쪽 눈 표시

        # 왼쪽 눈 깜빡임 감지
        if abs(left_eye[0].y - left_eye[1].y) < 0.005:
            eye_closed = True

        # 눈 깜빡임 로직
        if eye_closed and (last_blink_time is None or time.time() - last_blink_time > BLINK_DELAY):
            blink_count += 1
            last_blink_time = time.time()  # 마지막 깜빡임 시간 기록

    # 두 번 연속 깜빡임 감지 시 헬멧 인식 활성화
    if blink_count >= 2:
        helmet_detection_mode = True
        blink_count = 0  # 깜빡임 카운트 초기화

    if helmet_detection_mode and not authentication_completed:
        # YOLO로 헬멧 탐지
        results = model.predict(source=frame, save=False, show=False)

        for box in results[0].boxes:
            cls = int(box.cls)
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])

            # 바운딩 박스 그리기
            if cls == 1:  # With-helmet
                color = (0, 255, 0)  # 초록색
                label = "With helmet"
                if helmet_detected_time is None:
                    helmet_detected_time = time.time()  # 헬멧 감지 시작 시간 기록
                elif time.time() - helmet_detected_time >= 3 and qr_displayed_time is None:
                    qr_displayed_time = time.time()  # QR 코드 표시 시작 시간
                    cv2.imshow("Scan QR Code", qr_image)  # QR 코드 표시
            elif cls == 2:  # With cap
                color = (0, 255, 255)  # 노란색
                label = "With cap"
                helmet_detected_time = None  # 헬멧과 구분하기 위해 초기화
            elif cls == 0:  # No-helmet
                color = (0, 0, 255)  # 빨간색
                label = "No helmet"
                helmet_detected_time = None

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(
                frame,
                f"{label} ({conf:.2f})",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                color,
                2,
            )

        # QR 코드 표시 시간 확인
        if qr_displayed_time:
            # Streamlit 파일 확인
            if os.path.exists(streamlit_file_path):
                with open(streamlit_file_path, 'r') as file:
                    content = file.read().strip()
                    if content == "user":
                        authentication_complete_time = time.time()  # 인증 완료 시작 시간 기록
                        authentication_completed = True  # 인증 완료 상태 설정
                        qr_displayed_time = None
                        cv2.destroyWindow("Scan QR Code")  # QR 창 닫기

    # 인증 완료 후 헬멧/모자 상태 확인
    if authentication_completed:
        results = model.predict(source=frame, save=False, show=False)
        helmet_worn = False

        for box in results[0].boxes:
            cls = int(box.cls)
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])

            if cls == 1:  # With-helmet
                helmet_worn = True
                color = (0, 255, 0)  # 초록색
                label = "With helmet"
            elif cls == 2:  # With cap
                color = (0, 255, 255)  # 노란색
                label = "With cap"
            elif cls == 0:  # No-helmet
                color = (0, 0, 255)  # 빨간색
                label = "No helmet"

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(
                frame,
                f"{label} ({conf:.2f})",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                color,
                2,
            )

        if helmet_worn:
            cv2.putText(frame, "You're wearing a helmet!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)
        else:
            cv2.putText(frame, "Please wear a helmet!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)

    # 결과 화면 표시
    cv2.imshow("Helmet Detection", frame)

    # 'q' 키로 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


## 최종..

In [ ]:
import cv2
import time
import mediapipe as mp
from ultralytics import YOLO
import os

# YOLO 모델 로드
model = YOLO(r"C:\Users\user\Documents\AI_project_clone\helmet_detection\helmet_detection_model\weights\best.pt")

# Mediapipe Face Mesh 초기화
face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)

# QR 코드 이미지 경로
qr_image_path = r"C:\Users\user\Documents\AI_project_clone\사용자인증.jpeg"
qr_image = cv2.imread(qr_image_path)  # QR 코드 이미지 로드
qr_image = cv2.resize(qr_image, (300, 300))

# 다운로드된 Streamlit 파일 경로
streamlit_file_path = r"C:\Users\user\Downloads\streamlit_download.txt"

# 웹캠 열기
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# 눈 깜빡임 감지 상태 변수 초기화
blink_count = 0  # 깜빡임 횟수
last_blink_time = None  # 마지막 깜빡임 시간
BLINK_DELAY = 2  # 눈 깜빡임 간격 제한 (초)

# 헬멧 감지 상태 변수 초기화
helmet_detection_mode = False
helmet_detected_time = None
qr_displayed_time = None
authentication_complete_time = None
authentication_completed = False  # 인증 완료 상태 플래그

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    frame = cv2.flip(frame, 1)  # 좌우 반전
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Mediapipe로 얼굴 인식 및 눈 깜빡임 감지
    output = face_mesh.process(rgb_frame)
    landmark_points = output.multi_face_landmarks
    frame_h, frame_w, _ = frame.shape

    eye_closed = False

    if landmark_points:
        landmarks = landmark_points[0].landmark
        left_eye = [landmarks[145], landmarks[159]]
        for landmark in left_eye:
            x = int(landmark.x * frame_w)
            y = int(landmark.y * frame_h)
            cv2.circle(frame, (x, y), 3, (0, 255, 255))  # 왼쪽 눈 표시

        # 왼쪽 눈 깜빡임 감지
        if abs(left_eye[0].y - left_eye[1].y) < 0.005:
            eye_closed = True

        # 눈 깜빡임 로직
        if eye_closed and (last_blink_time is None or time.time() - last_blink_time > BLINK_DELAY):
            blink_count += 1
            last_blink_time = time.time()  # 마지막 깜빡임 시간 기록

    # 두 번 연속 깜빡임 감지 시 헬멧 인식 활성화
    if blink_count >= 2:
        helmet_detection_mode = True
        blink_count = 0  # 깜빡임 카운트 초기화

    if not helmet_detection_mode:
        cv2.putText(frame, "Blink twice to start helmet detection", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 2)
        
    if helmet_detection_mode and not authentication_completed:
        # YOLO로 헬멧 탐지
        results = model.predict(source=frame, save=False, show=False)

        for box in results[0].boxes:
            cls = int(box.cls)
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])

            # 바운딩 박스 그리기
            if cls == 1:  # With-helmet
                color = (0, 255, 0)  # 초록색
                label = "With helmet"
                if helmet_detected_time is None:
                    helmet_detected_time = time.time()  # 헬멧 감지 시작 시간 기록
                elif time.time() - helmet_detected_time >= 3 and qr_displayed_time is None:
                    qr_displayed_time = time.time()  # QR 코드 표시 시작 시간
                    cv2.imshow("Scan QR Code", qr_image)  # QR 코드 표시
            elif cls == 2:  # With cap
                color = (0, 255, 255)  # 노란색
                label = "With cap"
                helmet_detected_time = None  # 헬멧과 구분하기 위해 초기화
            elif cls == 0:  # No-helmet
                color = (0, 0, 255)  # 빨간색
                label = "No helmet"
                helmet_detected_time = None

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(
                frame,
                f"{label} ({conf:.2f})",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                color,
                2,
            )

        # QR 코드 표시 시간 확인
        if qr_displayed_time:
            # Streamlit 파일 확인
            if os.path.exists(streamlit_file_path):
                with open(streamlit_file_path, 'r') as file:
                    content = file.read().strip()
                    if content == "user":
                        authentication_complete_time = time.time()  # 인증 완료 시작 시간 기록
                        authentication_completed = True  # 인증 완료 상태 설정
                        qr_displayed_time = None
                        cv2.destroyWindow("Scan QR Code")  # QR 창 닫기

    # 인증 완료 후 헬멧/모자 상태 확인
    if authentication_completed:
        results = model.predict(source=frame,conf =0.4, save=False, show=False)
        helmet_worn = False

        for box in results[0].boxes:
            cls = int(box.cls)
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])

            if cls == 1:  # With-helmet
                helmet_worn = True
                color = (0, 255, 0)  # 초록색
                label = "With helmet"
            elif cls == 2:  # With cap
                color = (0, 255, 255)  # 노란색
                label = "With cap"
            elif cls == 0:  # No-helmet
                color = (0, 0, 255)  # 빨간색
                label = "No helmet"

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(
                frame,
                f"{label} ({conf:.2f})",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                color,
                2,
            )

        if helmet_worn:
            cv2.putText(frame, "You're wearing a helmet!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 3)
        else:
            cv2.putText(frame, "Please wear a helmet!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)

    # 결과 화면 표시
    cv2.imshow("Helmet Detection", frame)

    # 'q' 키로 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


## 진짜 최종

In [ ]:
import cv2
import time
import mediapipe as mp
from ultralytics import YOLO
import os

# YOLO 모델 로드
model = YOLO(r"C:\Users\user\Documents\AI_project_clone\helmet_detection\helmet_detection_model\weights\best.pt")

# Mediapipe Face Mesh 초기화
face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)

# QR 코드 이미지 경로
qr_image_path = r"C:\Users\user\Documents\AI_project_clone\사용자인증.jpeg"
qr_image = cv2.imread(qr_image_path)  # QR 코드 이미지 로드
qr_image = cv2.resize(qr_image, (300, 300))

# 다운로드된 Streamlit 파일 경로
streamlit_file_path = r"C:\Users\user\Downloads\streamlit_download.txt"

# 웹캠 열기
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# 눈 깜빡임 감지 상태 변수 초기화
blink_count = 0  # 깜빡임 횟수
last_blink_time = None  # 마지막 깜빡임 시간
BLINK_DELAY = 2  # 눈 깜빡임 간격 제한 (초)

# 헬멧 감지 상태 변수 초기화
helmet_detection_mode = False
helmet_detected_time = None
qr_displayed_time = None
authentication_complete_time = None
authentication_completed = False  # 인증 완료 상태 플래그

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    frame = cv2.flip(frame, 1)  # 좌우 반전
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Mediapipe로 얼굴 인식 및 눈 깜빡임 감지
    output = face_mesh.process(rgb_frame)
    landmark_points = output.multi_face_landmarks
    frame_h, frame_w, _ = frame.shape

    eye_closed = False

    if landmark_points:
        landmarks = landmark_points[0].landmark
        left_eye = [landmarks[145], landmarks[159]]
        for landmark in left_eye:
            x = int(landmark.x * frame_w)
            y = int(landmark.y * frame_h)
            cv2.circle(frame, (x, y), 3, (0, 255, 255))  # 왼쪽 눈 표시

        # 왼쪽 눈 깜빡임 감지
        if abs(left_eye[0].y - left_eye[1].y) < 0.005:
            eye_closed = True

        # 눈 깜빡임 로직
        if eye_closed and (last_blink_time is None or time.time() - last_blink_time > BLINK_DELAY):
            blink_count += 1
            last_blink_time = time.time()  # 마지막 깜빡임 시간 기록

    # 두 번 연속 깜빡임 감지 시 헬멧 인식 활성화
    if blink_count >= 2:
        helmet_detection_mode = True
        blink_count = 0  # 깜빡임 카운트 초기화

    # 상태에 따라 메시지 설정
    if helmet_detection_mode and not authentication_completed:
        cv2.putText(frame, "Helmet detection mode", (50, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)
    elif not helmet_detection_mode:
        cv2.putText(frame, "Blink twice to start helmet detection", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 2)

    if helmet_detection_mode and not authentication_completed:
        # YOLO로 헬멧 탐지
        results = model.predict(source=frame, save=False, show=False)

        helmet_detected = False
        no_helmet_detected = False

        for box in results[0].boxes:
            cls = int(box.cls)
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])

            if cls == 1:  # With-helmet
                helmet_detected = True
                color = (0, 255, 0)  # 초록색
                label = "With helmet"
                if helmet_detected_time is None:
                    helmet_detected_time = time.time()  # 헬멧 감지 시작 시간 기록
                elif time.time() - helmet_detected_time >= 3 and qr_displayed_time is None:
                    qr_displayed_time = time.time()  # QR 코드 표시 시작 시간
                    cv2.imshow("Scan QR Code", qr_image)  # QR 코드 표시
            elif cls == 0:  # No-helmet
                no_helmet_detected = True
                color = (0, 0, 255)  # 빨간색
                label = "No helmet"
                helmet_detected_time = None
            elif cls == 2:  # With cap
                color = (0, 255, 255)  # 노란색
                label = "With cap"
                helmet_detected_time = None

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(
                frame,
                f"{label} ({conf:.2f})",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                color,
                2,
            )

        if qr_displayed_time:
            if os.path.exists(streamlit_file_path):
                try:
                    with open(streamlit_file_path, 'r') as file:
                        content = file.read().strip()
                        if content == "0":
                            authentication_complete_time = time.time()  # 인증 완료 시작 시간 기록
                            authentication_completed = True  # 인증 완료 상태 설정
                            qr_displayed_time = None
                            cv2.destroyWindow("Scan QR Code")  # QR 창 닫기
                except Exception as e:
                    print(f"Error reading file: {e}")

        # 메시지 상태 관리
        if no_helmet_detected:
            cv2.putText(frame, "Please wear a helmet", (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
        elif helmet_detected:
            cv2.putText(frame, "Helmet detected", (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

    # 인증 완료 후 헬멧 상태 확인
    if authentication_completed:
        results = model.predict(source=frame, conf=0.4, save=False, show=False)
        helmet_worn = False

        for box in results[0].boxes:
            cls = int(box.cls)
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])

            if cls == 1:  # With-helmet
                helmet_worn = True
                color = (0, 255, 0)  # 초록색
                label = "With helmet"
            elif cls == 2:  # With cap
                color = (0, 255, 255)  # 노란색
                label = "With cap"
            elif cls == 0:  # No-helmet
                color = (0, 0, 255)  # 빨간색
                label = "No helmet"

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(
                frame,
                f"{label} ({conf:.2f})",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                color,
                2,
            )

        # 헬멧 착용 상태에 따른 메시지
        if helmet_worn:
            cv2.putText(frame, "You're wearing a helmet!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 3)
        else:
            cv2.putText(frame, "Please wear a helmet!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)

    
    # 결과 화면 표시
    cv2.imshow("Helmet Detection", frame)

    # 'q' 키로 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# 진짜진짜 최종25-01-22

In [ ]:
import cv2
import time
import mediapipe as mp
from ultralytics import YOLO
import os

# YOLO 모델 로드
model = YOLO(r"C:\Users\user\Documents\AI_project_clone\helmet_detection\helmet_detection_model\weights\best.pt")

# Mediapipe Face Mesh 초기화
face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)

# QR 코드 이미지 경로
qr_image_path = r"C:\Users\user\Documents\AI_project_clone\사용자인증.jpeg"
qr_image = cv2.imread(qr_image_path)  # QR 코드 이미지 로드
qr_image = cv2.resize(qr_image, (300, 300))

# 다운로드된 Streamlit 파일 경로
streamlit_file_path = r"C:\Users\user\Downloads\streamlit_download.txt"

# 웹캠 열기
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# 눈 깜빡임 감지 상태 변수 초기화
blink_count = 0  # 깜빡임 횟수
last_blink_time = None  # 마지막 깜빡임 시간
BLINK_DELAY = 2  # 눈 깜빡임 간격 제한 (초)
# 헬멧 감지 상태 변수 초기화
helmet_detected_time = None #헬멧 감지 시간
qr_displayed_time = None    #qr창 표시 시간
authentication_complete_time = None
authentication_completed = False  # 인증 완료 상태 플래그

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # 좌우 반전
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Mediapipe로 얼굴 인식 및 눈 깜빡임 감지
    output = face_mesh.process(rgb_frame)   #RGB로 변환된 프레임에서 얼굴 랜드마크 추출
    landmark_points = output.multi_face_landmarks
    frame_h, frame_w, _ = frame.shape

    eye_closed = False

    if landmark_points:
        landmarks = landmark_points[0].landmark
        left_eye = [landmarks[145], landmarks[159]]
        for landmark in left_eye:
            x = int(landmark.x * frame_w)
            y = int(landmark.y * frame_h)
            cv2.circle(frame, (x, y), 3, (0, 255, 255))  # 왼쪽 눈 표시

        # 왼쪽 눈 깜빡임 감지
        if abs(left_eye[0].y - left_eye[1].y) < 0.005:
            eye_closed = True

        # 눈 깜빡임 로직
        if eye_closed and (last_blink_time is None or time.time() - last_blink_time > BLINK_DELAY):
            blink_count += 1
            last_blink_time = time.time()  # 마지막 깜빡임 시간 기록

    # YOLO로 헬멧 탐지
    results = model.predict(source=frame, save=False, show=False)
    helmet_detected = False
    no_helmet_detected = True
    only_helmet_detected = False
    
    # YOLO 결과 처리
    if results[0].boxes:  # YOLO가 객체를 감지한 경우
        no_helmet_detected = False  # 객체가 감지되면 초기값을 업데이트

        for box in results[0].boxes:
            cls = int(box.cls)
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])

            if cls == 1 and conf > 0.5:  # With-helmet
                helmet_detected = True
                color = (0, 255, 0)  # 초록색
                label = "With helmet"
            elif cls == 0 and conf > 0.5:  # No-helmet
                no_helmet_detected = True
                color = (0, 0, 255)  # 빨간색
                label = "No helmet"
            elif cls == 2 and conf > 0.5:  # With cap
                color = (0, 255, 255)  # 노란색
                label = "With cap"
            elif cls == 3 and conf > 0.5:  # Only helmet
                only_helmet_detected = True
                color = (255, 0, 0)  # 파란색
                label = "Only helmet"

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(
                frame,
                f"{label} ({conf:.2f})",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                color,
                2,
            )

    # 메시지 출력 (우선 순위 설정)
    if not authentication_completed:  # 인증 완료되지 않은 경우에만 실행
        if no_helmet_detected:
            cv2.putText(frame, "Please wear a helmet", (50, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
        elif only_helmet_detected:
            cv2.putText(frame, "Please wear a full helmet!", (50, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 0), 3)
        elif helmet_detected:
            if blink_count < 2:  # 윙크가 아직 두 번 감지되지 않은 경우
                cv2.putText(frame, "Blink twice to start helmet detection", 
                            (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 2)
            elif blink_count >= 2:  # 윙크가 두 번 감지된 경우
                if qr_displayed_time is None:  # QR 창이 표시된 적 없을 때만
                    qr_displayed_time = time.time()  # QR 코드 표시 시작 시간
                    cv2.imshow("Scan QR Code", qr_image)  # QR 코드 표시
                    blink_count = 0  # 깜빡임 초기화
        elif qr_displayed_time:  # QR 창이 닫혔는지 확인
            if cv2.getWindowProperty("Scan QR Code", cv2.WND_PROP_VISIBLE) < 1:
                cv2.imshow("Scan QR Code", qr_image)  # QR 코드 다시 표시


    # QR 코드 인증 완료 확인
    if qr_displayed_time and not authentication_completed:
        if os.path.exists(streamlit_file_path):
            try:
                with open(streamlit_file_path, 'r') as file:
                    content = file.read().strip()
                    if content == "0":
                        authentication_completed = True  # 인증 완료 상태 설정
                        qr_displayed_time = None    #qr인증이 됐다면 다시 실행x
                        cv2.destroyWindow("Scan QR Code")  # QR 창 닫기

            except Exception as e:
                print(f"Error reading file: {e}")

    # 인증 완료 상태 메시지
    if authentication_completed:
        cv2.putText(frame, "Authentication Complete!", (50, 20), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 3)

        results = model.predict(source=frame, conf=0.4, save=False, show=False)
        helmet_worn = False
        only_helmet_detected = False

        for box in results[0].boxes:
            cls = int(box.cls)
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])

            if cls == 1 and conf > 0.5:  # With-helmet
                helmet_worn = True
                color = (0, 255, 0)  # 초록색
                label = "With helmet"
            elif cls == 2 and conf > 0.5:  # With cap
                color = (0, 255, 255)  # 노란색
                label = "With cap"
            elif cls == 0:  # No-helmet
                color = (0, 0, 255)  # 빨간색
                label = "No helmet"
            elif cls == 3 and conf > 0.5:  # Only helmet
                only_helmet_detected = True
                color = (255, 0, 0)  # 파란색
                label = "Only helmet"

            # 바운딩 박스와 레이블 표시
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(
                frame,
                f"{label} ({conf:.2f})",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                color,
                2,
            )

        # 헬멧 착용 상태에 따른 메시지
        if helmet_worn:
            cv2.putText(frame, "You're wearing a helmet!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 3)
        elif only_helmet_detected:
            cv2.putText(frame, "Please wear a full helmet!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 0, 0), 3)
        else:
            cv2.putText(frame, "Please wear a helmet!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)

    # 결과 화면 표시
    cv2.imshow("Helmet Detection", frame)

    # 'q' 키로 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## qr인식후 탐지기능 끄기

In [ ]:
import cv2
import time
import mediapipe as mp
from ultralytics import YOLO
import os

# YOLO 모델 로드
model = YOLO(r"C:\Users\user\Documents\AI_project_clone\helmet_detection\helmet_detection_model\weights\best.pt")

# Mediapipe Face Mesh 초기화
face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)

# QR 코드 이미지 경로
qr_image_path = r"C:\Users\user\Documents\AI_project_clone\사용자인증.jpeg"
qr_image = cv2.imread(qr_image_path)  # QR 코드 이미지 로드
qr_image = cv2.resize(qr_image, (300, 300))

# 다운로드된 Streamlit 파일 경로
streamlit_file_path = r"C:\Users\user\Downloads\streamlit_download.txt"

# 웹캠 열기
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# 눈 깜빡임 감지 상태 변수 초기화
blink_count = 0  # 깜빡임 횟수
last_blink_time = None  # 마지막 깜빡임 시간
BLINK_DELAY = 2  # 눈 깜빡임 간격 제한 (초)

# 헬멧 감지 상태 변수 초기화
helmet_detected_time = None
qr_displayed_time = None
authentication_complete_time = None
authentication_completed = False  # 인증 완료 상태 플래그

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    frame = cv2.flip(frame, 1)  # 좌우 반전
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Mediapipe로 얼굴 인식 및 눈 깜빡임 감지
    output = face_mesh.process(rgb_frame)
    landmark_points = output.multi_face_landmarks
    frame_h, frame_w, _ = frame.shape

    eye_closed = False

    if landmark_points:
        landmarks = landmark_points[0].landmark
        left_eye = [landmarks[145], landmarks[159]]
        for landmark in left_eye:
            x = int(landmark.x * frame_w)
            y = int(landmark.y * frame_h)
            cv2.circle(frame, (x, y), 3, (0, 255, 255))  # 왼쪽 눈 표시

        # 왼쪽 눈 깜빡임 감지
        if abs(left_eye[0].y - left_eye[1].y) < 0.005:
            eye_closed = True

        # 눈 깜빡임 로직
        if eye_closed and (last_blink_time is None or time.time() - last_blink_time > BLINK_DELAY):
            blink_count += 1
            last_blink_time = time.time()  # 마지막 깜빡임 시간 기록

    # YOLO로 헬멧 탐지
    results = model.predict(source=frame, save=False, show=False)
    helmet_detected = False
    no_helmet_detected = True
    only_helmet_detected = False
    
    # YOLO 결과 처리
    if not authentication_completed:
        if results[0].boxes:  # YOLO가 객체를 감지한 경우
            no_helmet_detected = False  # 객체가 감지되면 초기값을 업데이트

            for box in results[0].boxes:
                cls = int(box.cls)
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])

                if cls == 1 and conf > 0.5:  # With-helmet
                    helmet_detected = True
                    color = (0, 255, 0)  # 초록색
                    label = "With helmet"
                elif cls == 0 and conf > 0.5:  # No-helmet
                    no_helmet_detected = True
                    color = (0, 0, 255)  # 빨간색
                    label = "No helmet"
                elif cls == 2 and conf > 0.5:  # With cap
                    color = (0, 255, 255)  # 노란색
                    label = "With cap"
                elif cls == 3 and conf > 0.3:  # Only helmet
                    only_helmet_detected = True
                    color = (255, 0, 0)  # 파란색
                    label = "Only helmet"

                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(
                    frame,
                    f"{label} ({conf:.2f})",
                    (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    color,
                    2,
                )

    # 메시지 출력 (우선 순위 설정)
    if not authentication_completed:  # 인증 완료되지 않은 경우에만 실행
        if no_helmet_detected:
            cv2.putText(frame, "Please wear a helmet", (50, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
        elif only_helmet_detected:
            cv2.putText(frame, "Please wear a full helmet!", (50, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 0), 3)
        elif helmet_detected:
            if blink_count < 2:  # 윙크가 아직 두 번 감지되지 않은 경우
                cv2.putText(frame, "Blink twice to start helmet detection", 
                            (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 2)
            elif blink_count >= 2:  # 윙크가 두 번 감지된 경우
                if qr_displayed_time is None:  # QR 창이 표시된 적 없을 때만
                    qr_displayed_time = time.time()  # QR 코드 표시 시작 시간
                    cv2.imshow("Scan QR Code", qr_image)  # QR 코드 표시
                    blink_count = 0  # 깜빡임 초기화
        elif qr_displayed_time:  # QR 창이 닫혔는지 확인
            if cv2.getWindowProperty("Scan QR Code", cv2.WND_PROP_VISIBLE) < 1:
                cv2.imshow("Scan QR Code", qr_image)  # QR 코드 다시 표시


    # QR 코드 인증 완료 확인
    if qr_displayed_time and not authentication_completed:
        if os.path.exists(streamlit_file_path):
            try:
                with open(streamlit_file_path, 'r') as file:
                    content = file.read().strip()
                    if content == "0":
                        authentication_completed = True  # 인증 완료 상태 설정
                        qr_displayed_time = None
                        cv2.destroyWindow("Scan QR Code")  # QR 창 닫기

            except Exception as e:
                print(f"Error reading file: {e}")

    # 인증 완료 상태 메시지
    if authentication_completed:
        cv2.putText(frame, "Authentication Complete!", (50, 20), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 3)

        results = model.predict(source=frame, conf=0.4, save=False, show=False)
        helmet_worn = False
        only_helmet_detected = False

        for box in results[0].boxes:
            cls = int(box.cls)
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])

            if cls == 1 and conf > 0.5:  # With-helmet
                helmet_worn = True
                color = (0, 255, 0)  # 초록색
                label = "With helmet"
            elif cls == 2 and conf > 0.4:  # With cap
                color = (0, 255, 255)  # 노란색
                label = "With cap"
            elif cls == 0 and conf > 0.6:  # No-helmet
                color = (0, 0, 255)  # 빨간색
                label = "No helmet"
            elif cls == 3 and conf > 0.3:  # Only helmet
                only_helmet_detected = True
                color = (255, 0, 0)  # 파란색
                label = "Only helmet"

            # 바운딩 박스와 레이블 표시
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(
                frame,
                f"{label} ({conf:.2f})",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                color,
                2,
            )

        # 헬멧 착용 상태에 따른 메시지
        if helmet_worn:
            cv2.putText(frame, "You're wearing a helmet!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 3)
        elif only_helmet_detected:
            cv2.putText(frame, "Please wear a full helmet!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 0, 0), 3)
        else:
            cv2.putText(frame, "Please wear a helmet!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)

    # 결과 화면 표시
    cv2.imshow("Helmet Detection", frame)

    # 'q' 키로 종료
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()